### SRM coverage analysis for p, ph mapping for 'phased' and unphased regions

This notebook is aimed at calculating the coverage of the genome using short read illumina data mapped with bwa againts primary contigs and primary contigs + haplotigs.

* calculate the coverage in 1000 bp sliding windows (200bp) for p and p+h mapping.
* calculate the coverage when mapping against primary contigs in regions with h overlap and without
* plot those out and see

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import re
from Bio import SeqIO
from Bio import SeqUtils
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil
from Bio.Seq import Seq
import pysam
from Bio import SearchIO
import json
import glob
import scipy.stats as stats
import statsmodels as sms
import statsmodels.sandbox.stats.multicomp
import distance
import seaborn as sns
from pybedtools import BedTool
import matplotlib
from sklearn.externals.joblib import Parallel, delayed
import itertools as it
import tempfile
from scipy.signal import argrelextrema

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [3]:
#Define some PATH variables
BASE_AA_PATH = '/home/benjamin/genome_assembly/Warrior/genome_v04'
COV_PATH = '/home/benjamin/genome_assembly/Warrior/COV'
SRM_PATH = os.path.join('/home/benjamin/genome_assembly/Warrior/SRM')

In [4]:
genome = 'DK_0911_v04'
mapping_project = "PRI_NTKN_DK0911_on_%s" % genome
pbam_short_fn = 'DK_0911_v03_p_ctg.bwamem.PRI_NTKN_DK0911.sam.sorted.bam'
phbam_short_fn = 'DK_0911_v03_ph_ctg.bwamem.PRI_NTKN_DK0911.sam.sorted.bam'
h_on_p_fn = '/home/benjamin/genome_assembly/Warrior/allele_analysis/v04/allele_analysis/DK_0911_v04_p_ctg.p_by_h_cov.gff'
threads = 6
OUT_PATH = os.path.join(COV_PATH, 'analysis', mapping_project)
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
TMP_PATH = os.path.join(OUT_PATH, 'tmp')
if not os.path.exists(TMP_PATH):
    os.makedirs(TMP_PATH)

Now parallelize the bedtools and samtools functions.
This requires the following:
* split the window bedfile in as many non-overlapping bedfiels as we have threads and return the file locations as in a list.
* a function that takes a two filenames bedfile window and bam file and does a subprocess using samtools 





In [5]:
def run_command(command):
    print('\nRunnning now!\n')
    print(command)
    output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
    print('\nDone\nWith ouput:\n%s' % output)
    print(output)

In [6]:
def hello_print(command_list):
    print(command_list)

In [7]:
def faidx_genome(genome_fn, contig_fn):
    '''Generates a genome file used for samtools and such if not already present.
    Input: 
    Abspath filename of output genome_file.
    Abspath filename for genome contig file.'''
    if not os.path.exists(genome_fn):
        sammtools_command = 'samtools faidx %s' % contig_fn
        run_command(samtools_command)
        genome_file_command = 'cat %s.fai | sort -k1,1n | cut -f 1,2 > %s' %(contig_fn, genome_fn)
        run_command(genome_file_command)
    else:
        print('%s already exists!' % genome_fn)

In [8]:
def sort_gff_file(gff_fn):
    '''Sorts gff file and returns the Abspath to the sorted gff file.'''
    gff3sport_pl = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/get_homologues/gff3sort/gff3sort.pl'
    if gff_fn.endswith('gff'):
        out_fn = gff_fn.replace('.gff', '.sorted.gff')
    elif gff_fn.endswith('gff3'):
        out_fn = gff_fn.replace('.gff3', '.sorted.gff3')
    else:
        print('Please make sure your gff %s file ends with gff or gff3' % gff_fn.split['/'][-1])
    sort_command = 'perl %s --precise %s > %s' %(gff3sport_pl, gff_fn, out_fn)
    run_command(sort_command)
    return out_fn

In [9]:
def bed_file_split(bed_fn, num_chunks, tmp_path = TMP_PATH):
    '''Splits a bedfile into even chunks of files plus the reads. Each file having the same number of lines.
    Input:
    bed file name abspath
    chunks to split into
    tmp_path where the split files get saved into.
    Returns:
    List of abspath filenames for split bed files.'''
    infilename = bed_fn
    number_of_lines = 0
    with open(infilename, 'rb') as infile:
        for x in infile:
            number_of_lines = number_of_lines + 1
    print('in_file_line_number:', number_of_lines)
    chunk_size = number_of_lines // num_chunks
    print('target chunk_size:', chunk_size)
    files = []
    
    previous = 0
    for i,y in enumerate([x for x in range(0, number_of_lines, chunk_size)]):
        if y == 0:
            continue
                    
        else:
            temp_fn = os.path.join(TMP_PATH, '%s_%s' % (os.path.basename(bed_fn), i-1))
            with open(temp_fn, 'w') as temp_file:
                with open(infilename, 'r') as infile:
                    for line_number, line in enumerate(infile):
                        if line_number < y and line_number >= previous:
                            print(line.rstrip(), file=temp_file)
                        else:
                            continue
            previous = y
            files.append(temp_fn)
    if y < number_of_lines:
        temp_fn = os.path.join(TMP_PATH, '%s_%s' % (os.path.basename(bed_fn), i))
        with open(temp_fn, 'w') as temp_file:
                with open(infilename, 'r') as infile:
                    for line_number, line in enumerate(infile):
                        if line_number >= y:
                            print(line.rstrip(), file=temp_file)
                        else:
                            continue
    files.append(temp_fn)
    return files

In [10]:
def samcov_file_split(basefile_name, chunks, tmp_path = TMP_PATH):
    """This function takes the samcov file and generates the split filenames necessary to run the 
    parallelized run_sam_bedcov."""
    chunk_base_name = os.path.join(tmp_path, os.path.basename(basefile_name))
    return_list = []
    for x in range(0, chunks):
        return_list.append('%s_%s' % (chunk_base_name, x))
    return return_list

In [11]:
def combine_samcov_files(file_list):
    """This function concatenates samcov files from a tmp folder into the folder below. And removes all tmp files."""
    command = 'cat'
    combined_file = os.path.basename(file_list[0])[:-2]
    combined_file_fn = os.path.join(os.path.dirname(file_list[0]),'..', combined_file)
    for file in file_list:
        command = command + ' %s' % file
    command = command + ' > %s'  % combined_file_fn
    run_command(command)

In [12]:
def remove_files(file_list):
    for file in file_list:
        os.remove(file)

In [13]:
def run_sam_bedcov(bed_fn, bam_fn, samcov_fn, parallel = False):
    """This function runst samtools bedcov. It also has the option to do this in parallel spliting up the bed_cov file in equal junks plus one.
    Input:
        bed file name as Abspath.
        bam file name as Abspath.
        Samcov file as Abspath.
        Option to run thinks in parllel. Set to False. Provide the number of threads you want to run -1.
    Output:
        It generate the samcov files and saves them to file."""
    command = 'samtools bedcov %s %s > %s' % (bed_fn, bam_fn, samcov_fn)
    if parallel != False and type(parallel) == int:
        split_bed_files = bed_file_split(bed_fn, parallel)
        split_samcov_files = samcov_file_split(samcov_fn, len(split_bed_files), tmp_path=TMP_PATH)
        bam_files = [bam_fn]*len(split_bed_files)
        n_threads = len(split_bed_files)
        Parallel(n_jobs=n_threads)(delayed(run_sam_bedcov)(bed, bam, sam) for bed, bam, sam in zip(split_bed_files, bam_files, split_samcov_files))
        combine_samcov_files(split_samcov_files)
        remove_files(split_samcov_files)
        remove_files(split_bed_files)
    else:
        run_command(command)

In [14]:
def diploid_coverage(df):
    """This function tries to define the 'diploid coverage' of a samcov dataframe in case of two major coverage peaks.
    In case of a single peak it will return its coverage.
    It checks if the dataframe contains high coverage outliers and removes those for the analysis.
    Input:
        Samcov dataframe. 
    Output:
        Location of diploid coverage peak for diploids.
        Location of haploid coverage peak in case of haploids.
    """
    #function that first checks if the dataframe if trimmed down
    if df.ave_cov.max() > 10 * df.ave_cov.mean():
        print("Reducing dataframe to quantile 0.99")
        q_up = 0.99
        q_down = 1 - 0.99
        low = x.quantile(q_down) - 1.5*(x.quantile(q_up) - x.quantile(q_down))
        high = x.quantile(q_up) + 1.5*(x.quantile(q_up) - x.quantile(q_down))
        df = df[(df.ave_cov >= low) & (df.ave_cov <= high)]
    cov_bin = pd.cut(df.ave_cov, bins=200)
    value_counts = cov_bin.value_counts().sort_index()
    value_counts_df = value_counts.reset_index()
    local_max = argrelextrema(value_counts.values, np.greater, order=10, mode ='wrap')[0]
    sub_df = value_counts_df.loc[local_max]
    sub_df['mids'] = sub_df['index'].apply(lambda x: float(x.mid))
    sub_df = sub_df[sub_df.ave_cov > sub_df.ave_cov.max()*0.1]
    #now comes the heuristic
    print('These are the options for the peaks that are greater than 10% of the max peak.')
    print(sub_df)
    if len(sub_df.mids) == 1:
        #print("step 1")
        return sub_df.mids[sub_df.mids.index[0]]
    elif len(sub_df.mids) == 2:
        #print('step 2')
        return sub_df.mids[sub_df.mids.index[1]]
    elif len(sub_df.mids) > 2:
        #drop everything at the left edge
        mids = sub_df.mids[np.array(sub_df['mids']) >0.1*df.ave_cov.mean()]
        if len(mids) == 1:
            #print('something')
            return mids[mids.index[0]]
            #continue
        elif len(mids) == 2:
            dvision = mids[mids.index[1]]/mids[mids.index[0]] 
            if 1.85 < dvision < 2.15:
                return mids[mids.index[1]]

In [15]:
def samcov_slurp(file_name, fil=True, low=0, high=1000000, quantile = False, norm = False):
    """Reads in a samcov file as a dataframe and can filter it by quantiles, high and low values.
    It can also normalize. Either by a given value or by the calculated diploid coverage if norm is set to true.
    If norm is set to false it will simply normalize by the overall mean coverage of the dataframe. This could be an issue
    for multi peak coverage plots."""
    samcov_header = ['contig', 'start', 'stop', 'total_cov']
    df = pd.read_csv(file_name, sep='\t', header=None, names=samcov_header)
    df['ave_cov'] = df.total_cov/(df.stop-df.start)
    #rounder = pd.Series([0,0,0,0,2], index = df.columns)
    df.ave_cov = df.ave_cov.round()
    
    if fil == True and quantile == False:
        df = df[(df.ave_cov >= low) & (df.ave_cov <= high)]
    elif fil == True and type(quantile) == float:
        x = df.ave_cov
        q_up = quantile
        q_down = 1 - quantile
        low = x.quantile(q_down) - 1.5*(x.quantile(q_up) - x.quantile(q_down))
        high = x.quantile(q_up) + 1.5*(x.quantile(q_up) - x.quantile(q_down))
        df = df[(df.ave_cov >= low) & (df.ave_cov <= high)]
    if 'p_mapping' in file_name:
        df = df[df.contig.str.startswith('pcontig')].copy()
    
    if norm == False:
        df['norm_cov'] = df['ave_cov']/df['ave_cov'].mean()
    elif type(norm) == int:
        df['norm_cov'] = df['ave_cov']/norm
    elif norm == True:
        calcuated_coverage = diploid_coverage(df)
        df['norm_cov'] = df['ave_cov']/calcuated_coverage
        print("This is the calculated diploid coverage: %i" % calcuated_coverage)
    return df

### Program starts here ###

In [16]:
#define some filenames for assembly files
p_fn = os.path.join(BASE_AA_PATH, '%s_p_ctg.fa' % genome)
ph_fn = os.path.join(BASE_AA_PATH,'%s_ph_ctg.fa' % genome)
p_genome_file_fn = os.path.join(BASE_AA_PATH, '%s_p_ctg.genome_file' % genome)
ph_genome_file_fn = os.path.join(BASE_AA_PATH, \
                                 '%s_ph_ctg.genome_file' % genome)
#and mapping bam files
pbam_fn = os.path.join(SRM_PATH, pbam_short_fn)
phbam_fn = os.path.join(SRM_PATH, phbam_short_fn)

In [17]:
#run some indexing and genome_file commands
faidx_genome(p_genome_file_fn, p_fn)
faidx_genome(ph_genome_file_fn, ph_fn)
h_on_p_sorted_fn = sort_gff_file(h_on_p_fn)

/home/benjamin/genome_assembly/Warrior/genome_v04/DK_0911_v04_p_ctg.genome_file already exists!
/home/benjamin/genome_assembly/Warrior/genome_v04/DK_0911_v04_ph_ctg.genome_file already exists!

Runnning now!

perl /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/get_homologues/gff3sort/gff3sort.pl --precise /home/benjamin/genome_assembly/Warrior/allele_analysis/v04/allele_analysis/DK_0911_v04_p_ctg.p_by_h_cov.gff > /home/benjamin/genome_assembly/Warrior/allele_analysis/v04/allele_analysis/DK_0911_v04_p_ctg.p_by_h_cov.sorted.gff

Done
With ouput:
b''
b''


In [18]:
#now make some window files
p_window_1k200_bed = BedTool().window_maker(g=p_genome_file_fn, w=1000, s=200)
ph_window_1k200_bed = BedTool().window_maker(g=ph_genome_file_fn, w=1000, s=200)
h_on_p_bed = BedTool(fn=h_on_p_sorted_fn)
h_on_p_bed_pos_1k200_bed = p_window_1k200_bed.intersect(h_on_p_bed)
h_on_p_bed_neg_1k200_bed = p_window_1k200_bed.subtract(h_on_p_bed)

In [19]:
#define some outfile names
p_window_1k200_fn = os.path.join(OUT_PATH, '%s_p_ctg.p_window_1k200.cov' % mapping_project)
ph_window_1k200_fn = os.path.join(OUT_PATH, '%s_ph_ctg.ph_window_1k200.cov' % mapping_project)
p_h_on_p_window_1k200_fn = os.path.join(OUT_PATH, \
                                        '%s_ctg.h_on_p.p_window_1k200.cov' %mapping_project)
p_h_not_on_p_window_1k200_fn = os.path.join(OUT_PATH, \
                                        '%s_p_ctg.h_not_on_p.p_window_1k200.cov' %mapping_project)

In [20]:
#save out the bed files for overlaps
p_window_1k200_bed.saveas(p_window_1k200_fn.replace('.cov', '.bed'))
ph_window_1k200_bed.saveas(ph_window_1k200_fn.replace('.cov','.bed'))
h_on_p_bed_pos_1k200_bed.saveas(p_h_on_p_window_1k200_fn.replace('.cov', '.bed'))
h_on_p_bed_neg_1k200_bed.saveas(p_h_not_on_p_window_1k200_fn.replace('.cov', '.bed'))

<BedTool(/home/benjamin/genome_assembly/Warrior/COV/analysis/PRI_NTKN_DK0911_on_DK_0911_v04/PRI_NTKN_DK0911_on_DK_0911_v04_p_ctg.h_not_on_p.p_window_1k200.bed)>

In [21]:
bed_list = [p_window_1k200_fn.replace('.cov', '.bed'), ph_window_1k200_fn.replace('.cov', '.bed'),\
            p_h_on_p_window_1k200_fn.replace('.cov', '.bed'), p_h_not_on_p_window_1k200_fn.replace('.cov', '.bed'),\
           p_h_on_p_window_1k200_fn.replace('.cov', '.bed'), p_h_not_on_p_window_1k200_fn.replace('.cov', '.bed')]
bam_list = [pbam_fn, phbam_fn, phbam_fn, phbam_fn,pbam_fn, pbam_fn]
samcov_list = [p_window_1k200_fn.replace('.cov', '.p_mapping.samcov'), ph_window_1k200_fn.replace('.cov', '.ph_mapping.samcov'),\
              p_h_on_p_window_1k200_fn.replace('.cov', '.ph_mapping.samcov'), p_h_not_on_p_window_1k200_fn.replace('.cov', '.ph_mapping.samcov'),\
              p_h_on_p_window_1k200_fn.replace('.cov', '.p_mapping.samcov'), p_h_not_on_p_window_1k200_fn.replace('.cov', '.p_mapping.samcov')]

In [ ]:
for bed, bam, sam in zip(bed_list, bam_list, samcov_list):
    run_sam_bedcov(bed, bam, sam, parallel=threads)

in_file_line_number: 372183
target chunk_size: 62030

Runnning now!

Runnning now!

Runnning now!

Runnning now!



Runnning now!

samtools bedcov /home/benjamin/genome_assembly/Warrior/COV/analysis/PRI_NTKN_DK0911_on_DK_0911_v04/tmp/PRI_NTKN_DK0911_on_DK_0911_v04_p_ctg.p_window_1k200.bed_0 /home/benjamin/genome_assembly/Warrior/SRM/DK_0911_v03_p_ctg.bwamem.PRI_NTKN_DK0911.sam.sorted.bam > /home/benjamin/genome_assembly/Warrior/COV/analysis/PRI_NTKN_DK0911_on_DK_0911_v04/tmp/PRI_NTKN_DK0911_on_DK_0911_v04_p_ctg.p_window_1k200.p_mapping.samcov_0

Runnning now!

Runnning now!
samtools bedcov /home/benjamin/genome_assembly/Warrior/COV/analysis/PRI_NTKN_DK0911_on_DK_0911_v04/tmp/PRI_NTKN_DK0911_on_DK_0911_v04_p_ctg.p_window_1k200.bed_1 /home/benjamin/genome_assembly/Warrior/SRM/DK_0911_v03_p_ctg.bwamem.PRI_NTKN_DK0911.sam.sorted.bam > /home/benjamin/genome_assembly/Warrior/COV/analysis/PRI_NTKN_DK0911_on_DK_0911_v04/tmp/PRI_NTKN_DK0911_on_DK_0911_v04_p_ctg.p_window_1k200.p_mapping.samcov_1

In [ ]:
for sam_file in samcov_list:
    print(os.path.exists(sam_file))

In [ ]:
#now read in the samtools bedcov files
p_window_1k200_samcov_df = samcov_slurp(samcov_list[1] +'_1', quantile = 0.99, norm = True)

In [ ]:
p_window_1k200_samcov_df.norm_cov.plot.hist(bins=200)

In [ ]:
#now add also the p_mapping p wo and w h overlap
p_h_on_p_window_1k200_samcov_p_mapping_df = samcov_slurp(\
                        p_h_on_p_window_1k200_fn.replace('.cov', '.p_mapping.samcov'))
p_h_not_on_p_window_1k200_samcov_p_mapping_df = samcov_slurp(\
                        p_h_not_on_p_window_1k200_fn.replace('.cov', '.p_mapping.samcov'))
p_h_on_p_window_1k200_samcov_p_mapping_uf_df = samcov_slurp(\
                        p_h_on_p_window_1k200_fn.replace('.cov', '.p_mapping.samcov'),fil=False)
p_h_not_on_p_window_1k200_samcov_p_mapping_uf_df = samcov_slurp(\
                        p_h_not_on_p_window_1k200_fn.replace('.cov', '.p_mapping.samcov'),fil=False)

In [ ]:
p_window_1k200_samcov_uf_df = samcov_slurp(p_window_1k200_fn.replace('.cov', '.samcov'),fil=False)
ph_window_1k200_samcov_uf_df = samcov_slurp(ph_window_1k200_fn.replace('.cov', '.samcov'),fil=False)
p_h_on_p_window_1k200_samcov_uf_df = samcov_slurp(p_h_on_p_window_1k200_fn.replace('.cov', '.samcov'),fil=False)
p_h_not_on_p_window_1k200_samcov_uf_df = samcov_slurp(p_h_not_on_p_window_1k200_fn.replace('.cov', '.samcov'),fil=False)

In [ ]:
#histogram of 
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 20}
label_config_x = {'fontsize'            : 'large',
      'verticalalignment'   : 'top',
      'horizontalalignment' : 'center'
      }
label_config_y = {'fontsize'            : 'large',
      'verticalalignment'   : 'bottom',
      'horizontalalignment' : 'center'
      }
matplotlib.rc('font', **font)

In [ ]:
#now do the figure
fig, ax1 = plt.subplots(3, 2, figsize=(25,20), sharex='all')
#set style
sns.set_style("ticks")
sns.despine()
pallete = sns.color_palette('colorblind')


ax1[0,0].hist(p_window_1k200_samcov_df.ave_cov, bins=100, color=pallete[0])
ax1[0,0].text(60, 22000, 'mean cov = %i\nmedian cov = %i' % \
             (p_window_1k200_samcov_uf_df.ave_cov.mean(),\
              p_window_1k200_samcov_uf_df.ave_cov.median() ))
ax1[0,0].text(60, 35000, 'ref = primary contigs')
ax1[0,0].plot([60, 100], [33000, 33000], lw =4, color = pallete[0])
ax1[0,0].text(55, 32000, 'p')
ax1[0,0].set_xlim([0,115])

#now plot the ph mapping with primeray mapping only
ph_p_samcov_df = \
ph_window_1k200_samcov_df[ph_window_1k200_samcov_df.contig.str.startswith('p')] 
ph_p_samcov_un_df = \
ph_window_1k200_samcov_uf_df[ph_window_1k200_samcov_uf_df.contig.str.startswith('p')]

ax1[1,0].hist(ph_p_samcov_df.ave_cov, bins=100, color=pallete[1])
ax1[1,0].text(60, 40000, 'mean cov = %i\nmedian cov = %i' % \
             (ph_p_samcov_un_df.ave_cov.mean(),\
              ph_p_samcov_un_df.ave_cov.median() ))
ax1[1,0].text(60, 75000, 'ref = primary contigs + haplotigs')
ax1[1,0].plot([60, 100], [68000, 68000], lw =4, color = pallete[1])
ax1[1,0].plot([60, 70], [63000, 63000], lw =4, color = 'k')
ax1[1,0].plot([80, 100], [63000, 63000], lw =4, color = 'k')
ax1[1,0].text(55, 62000, 'p\nh')
ax1[1,0].set_xlim([0,115])


#now plot the ph mapping with haplotig mapping only
ph_h_samcov_df = \
ph_window_1k200_samcov_df[ph_window_1k200_samcov_df.contig.str.startswith('h')] 
ph_h_samcov_un_df = \
ph_window_1k200_samcov_uf_df[ph_window_1k200_samcov_uf_df.contig.str.startswith('h')]

ax1[2,0].hist(ph_h_samcov_df.ave_cov, bins=100, color=pallete[2])
ax1[2,0].text(60, 40000, 'mean cov = %i\nmedian cov = %i' % \
             (ph_h_samcov_un_df.ave_cov.mean(),\
              ph_h_samcov_un_df.ave_cov.median() ))
ax1[2,0].text(60, 75000, 'ref = primary contigs + haplotigs')
ax1[2,0].plot([60, 100], [68000, 68000], lw =4, color = 'k')
ax1[2,0].plot([60, 70], [63000, 63000], lw =4, color = pallete[2])
ax1[2,0].plot([80, 100], [63000, 63000], lw =4, color = pallete[2])
ax1[2,0].text(55, 62000, 'p\nh')
ax1[2,0].set_xlim([0,115])


#now plot the ph mapping with primaries with overlaps
ax1[1,1].hist(p_h_on_p_window_1k200_samcov_df.ave_cov, bins=100, color=pallete[3])
ax1[1,1].text(60, 32000, 'mean cov = %i\nmedian cov = %i' % \
             (p_h_on_p_window_1k200_samcov_uf_df.ave_cov.mean(),\
              p_h_on_p_window_1k200_samcov_uf_df.ave_cov.median() ))
ax1[1,1].text(60, 58000, 'ref = primary contigs + haplotigs')
ax1[1,1].plot([60, 70], [52000, 52000], lw =4, color = pallete[3])
ax1[1,1].plot([80, 100], [52000, 52000], lw =4, color = pallete[3])
ax1[1,1].plot([70, 80], [52000, 52000], lw =4, color = 'k')
ax1[1,1].plot([60, 70], [48000, 48000], lw =4, color = 'k')
ax1[1,1].plot([80, 100], [48000, 48000], lw =4, color = 'k')
ax1[1,1].text(55, 47000, 'p\nh')
ax1[1,1].set_xlim([0,115])

#now plot the ph mapping with primaries with overlaps
ax1[2,1].hist(p_h_not_on_p_window_1k200_samcov_df.ave_cov, bins=100, color=pallete[4])
ax1[2,1].text(60, 15000, 'mean cov = %i\nmedian cov = %i' % \
             (p_h_not_on_p_window_1k200_samcov_uf_df.ave_cov.mean(),\
              p_h_not_on_p_window_1k200_samcov_uf_df.ave_cov.median() ))
ax1[2,1].text(60, 28000, 'ref = primary contigs + haplotigs')
ax1[2,1].plot([60, 70], [26000, 26000], lw =4, color = 'k')
ax1[2,1].plot([80, 100], [26000, 26000], lw =4, color = 'k')
ax1[2,1].plot([70, 80], [26000, 26000], lw =4, color = pallete[4])
ax1[2,1].plot([60, 70], [24000, 24000], lw =4, color = 'k')
ax1[2,1].plot([80, 100], [24000, 24000], lw =4, color = 'k')
ax1[2,1].text(55, 23500, 'p\nh')

ax1[2,1].set_xlim([0,115])

#add the labels
fig.text(0.06, 0.5, 'Frequency', ha='center', va='center', rotation='vertical')
fig.text(0.5, 0.09, 'Fold Coverage',ha='center', va='center')

#make plot [0,1] disappear
ax1[0,1].axis('off')
text = '''
Legend:\n
ref == reference used for Illumina short read mapping
p == primary contigs
h == haplotigs
color decodes genome regions that are analyzed for coverage
e.g. pink indicates regions of primary contigs that have
an overlapping haplotig
'''
ax1[0,1].text(0, 0.5, text)
out_fn = os.path.join(OUT_PATH, '%s_SRM_coverage.png' % genome)
#fig.savefig(out_fn, dpi=600)

In [ ]:
#now do the figure
fig, ax1 = plt.subplots(4, 2, figsize=(25,30), sharex='all')
#set style
sns.set_style("ticks")
sns.despine()
pallete = sns.color_palette('colorblind')


ax1[0,0].hist(p_window_1k200_samcov_df.ave_cov, bins=100, color=pallete[0])
ax1[0,0].text(200, 22000, 'mean cov = %i\nmedian cov = %i' % \
             (p_window_1k200_samcov_uf_df.ave_cov.mean(),\
              p_window_1k200_samcov_uf_df.ave_cov.median() ))
ax1[0,0].text(180, 35000, 'ref = primary contigs')
ax1[0,0].plot([200, 390], [33000, 33000], lw =4, color = pallete[0])
ax1[0,0].text(180, 32000, 'p')
ax1[0,0].set_xlim([0,400])

#add label
ax1[0,0].text(10,35000, 'A', fontsize=30)

#now add the primary mapping plots with p w h
ax1[0,1].hist(p_h_on_p_window_1k200_samcov_p_mapping_df.ave_cov, bins=100, color=pallete[3])
ax1[0,1].text(200, 22000, 'mean cov = %i\nmedian cov = %i' % \
             (p_h_on_p_window_1k200_samcov_p_mapping_uf_df.ave_cov.mean(),\
              p_h_on_p_window_1k200_samcov_p_mapping_uf_df.ave_cov.median() ))
ax1[0,1].text(180, 35000, 'ref = primary contigs')
ax1[0,1].plot([200, 250], [33000, 33000], lw =4, color = pallete[3])
ax1[0,1].plot([300, 390], [33000, 33000], lw =4, color = pallete[3])
ax1[0,1].plot([250, 300], [33000, 33000], lw =4, color = 'k')
ax1[0,1].plot([200, 250], [31000, 31000], lw =4, color = 'k')
ax1[0,1].plot([300, 390], [31000, 31000], lw =4, color = 'k')
ax1[0,1].text(180, 30000, 'p\nh')
ax1[0,1].set_xlim([0,400])

#add label
ax1[0,1].text(10,35000, 'B', fontsize=30)

#now add the primary mapping plots with p wo h
ax1[1,1].hist(p_h_not_on_p_window_1k200_samcov_p_mapping_df.ave_cov, bins=100, color=pallete[4])
ax1[1,1].text(200, 8000, 'mean cov = %i\nmedian cov = %i' % \
             (p_h_not_on_p_window_1k200_samcov_p_mapping_uf_df.ave_cov.mean(),\
              p_h_not_on_p_window_1k200_samcov_p_mapping_uf_df.ave_cov.median() ))
ax1[1,1].text(180, 18000, 'ref = primary contigs')
ax1[1,1].plot([200, 250], [16000, 16000], lw =4, color = 'k')
ax1[1,1].plot([300, 390], [16000, 16000], lw =4, color = 'k')
ax1[1,1].plot([250, 300], [16000, 16000], lw =4, color = pallete[4])
ax1[1,1].plot([200, 250], [15000, 15000], lw =4, color = 'k')
ax1[1,1].plot([300, 390], [15000, 15000], lw =4, color = 'k')
ax1[1,1].text(180, 14500, 'p\nh')
ax1[1,1].set_xlim([0,400])

#add label
ax1[1,1].text(10,17500, 'C', fontsize=30)

#now plot the ph mapping with primeray mapping only
ph_p_samcov_df = \
ph_window_1k200_samcov_df[ph_window_1k200_samcov_df.contig.str.startswith('p')] 
ph_p_samcov_un_df = \
ph_window_1k200_samcov_uf_df[ph_window_1k200_samcov_uf_df.contig.str.startswith('p')]

ax1[2,0].hist(ph_p_samcov_df.ave_cov, bins=100, color=pallete[1])
ax1[2,0].text(200, 40000, 'mean cov = %i\nmedian cov = %i' % \
             (ph_p_samcov_un_df.ave_cov.mean(),\
              ph_p_samcov_un_df.ave_cov.median() ))
ax1[2,0].text(180, 75000, 'ref = primary contigs + haplotigs')
ax1[2,0].plot([200, 390], [68000, 68000], lw =4, color = pallete[1])
ax1[2,0].plot([200, 250], [63000, 63000], lw =4, color = 'k')
ax1[2,0].plot([300, 390], [63000, 63000], lw =4, color = 'k')
ax1[2,0].text(180, 62000, 'p\nh')
ax1[2,0].set_xlim([0,400])

#add label
ax1[2,0].text(10,80000, 'D', fontsize=30)

#now plot the ph mapping with haplotig mapping only
ph_h_samcov_df = \
ph_window_1k200_samcov_df[ph_window_1k200_samcov_df.contig.str.startswith('h')] 
ph_h_samcov_un_df = \
ph_window_1k200_samcov_uf_df[ph_window_1k200_samcov_uf_df.contig.str.startswith('h')]

ax1[3,0].hist(ph_h_samcov_df.ave_cov, bins=100, color=pallete[2])
ax1[3,0].text(200, 40000, 'mean cov = %i\nmedian cov = %i' % \
             (ph_h_samcov_un_df.ave_cov.mean(),\
              ph_h_samcov_un_df.ave_cov.median() ))
ax1[3,0].text(180, 75000, 'ref = primary contigs + haplotigs')
ax1[3,0].plot([200, 390], [68000, 68000], lw =4, color = 'k')
ax1[3,0].plot([200, 250], [63000, 63000], lw =4, color = pallete[2])
ax1[3,0].plot([300, 390], [63000, 63000], lw =4, color = pallete[2])
ax1[3,0].text(180, 62000, 'p\nh')
ax1[3,0].set_xlim([0,400])
#add label
ax1[3,0].text(10,70000, 'F', fontsize=30)


#now plot the ph mapping with primaries with overlaps
ax1[2,1].hist(p_h_on_p_window_1k200_samcov_df.ave_cov, bins=100, color=pallete[3])
ax1[2,1].text(200, 32000, 'mean cov = %i\nmedian cov = %i' % \
             (p_h_on_p_window_1k200_samcov_uf_df.ave_cov.mean(),\
              p_h_on_p_window_1k200_samcov_uf_df.ave_cov.median() ))
ax1[2,1].text(180, 58000, 'ref = primary contigs + haplotigs')
ax1[2,1].plot([200, 250], [52000, 52000], lw =4, color = pallete[3])
ax1[2,1].plot([300, 390], [52000, 52000], lw =4, color = pallete[3])
ax1[2,1].plot([250, 300], [52000, 52000], lw =4, color = 'k')
ax1[2,1].plot([200, 250], [48000, 48000], lw =4, color = 'k')
ax1[2,1].plot([300, 390], [48000, 48000], lw =4, color = 'k')
ax1[2,1].text(180, 47000, 'p\nh')
ax1[2,1].set_xlim([0,400])

#add label
ax1[2,1].text(10,63000, 'E', fontsize=30)


#now plot the ph mapping with primaries with overlaps
ax1[3,1].hist(p_h_not_on_p_window_1k200_samcov_df.ave_cov, bins=100, color=pallete[4])
ax1[3,1].text(200, 15000, 'mean cov = %i\nmedian cov = %i' % \
             (p_h_not_on_p_window_1k200_samcov_uf_df.ave_cov.mean(),\
              p_h_not_on_p_window_1k200_samcov_uf_df.ave_cov.median() ))
ax1[3,1].text(180, 28000, 'ref = primary contigs + haplotigs')
ax1[3,1].plot([200, 250], [26000, 26000], lw =4, color = 'k')
ax1[3,1].plot([300, 390], [26000, 26000], lw =4, color = 'k')
ax1[3,1].plot([250, 300], [26000, 26000], lw =4, color = pallete[4])
ax1[3,1].plot([200, 250], [24000, 24000], lw =4, color = 'k')
ax1[3,1].plot([300, 390], [24000, 24000], lw =4, color = 'k')
ax1[3,1].text(180, 23500, 'p\nh')

ax1[3,1].set_xlim([0,400])

#add label
ax1[3,1].text(10,26000, 'G', fontsize=30)

#add the labels
fig.text(0.06, 0.5, 'Frequency', ha='center', va='center', rotation='vertical')
fig.text(0.5, 0.09, 'Fold Coverage',ha='center', va='center')

#make plot [0,1] disappear
ax1[1,0].axis('off')
text = '''
Legend:\n
ref == reference used for Illumina short read mapping
p == primary contigs
h == haplotigs
color decodes genome regions 
that are analyzed for coverage
e.g. pink indicates regions of primary contigs that have
an overlapping haplotig
'''
ax1[1,0].text(0, 0.5, text)
out_fn = os.path.join(OUT_PATH, 'SRM_coverage_v2.png')
#fig.savefig(out_fn, dpi=600)

In [ ]:
print(ax1[1,1])

In [ ]:
#add single label
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
plt.grid(False)
plt.xlabel("Fold coverage", {
      'fontsize'            : 20,
      'verticalalignment'   : 'top',
      'horizontalalignment' : 'center',
        'weight' : 'bold'
      })
plt.ylabel("Frequency")